1\. Write a function that converts number representation (bin<->dec<->hex)

In [2]:
def Converter_num(x):
    
    #Conversion from decimal to binary and viceversa
    bin1 = bin(x)
    print('Conversion from decimal to binary and viceversa')
    print('Binary representation of', x, ':', bin1)
    print('Decimal representation of', bin1, ':', int(bin1, 2),'\n')
    
    #Conversion from decimal to Hexadecimal and viceversa
    hex1 = hex(x)
    print('Conversion from decimal to Hexadecimal and viceversa')
    print('Hexadecimal representation of', x,':', hex1)
    print('Decimal representation of', hex1,':', int(hex1,16),'\n')
    
    #Conversion from Hexadecimal to binary
    bin1 = bin(int(hex1, 16))
    print('Conversion from Hexadecimal to binary')
    print('Binary representation of', hex1, ':', bin1,'\n')
    
    #Conversion from binary to Hexadecimal
    hex1 = hex(int(bin1, 2))
    print('Conversion from binary to Hexadecimal')
    print('Hexadecimal representation of', bin1, ':', hex1,'\n')
    
x = 26
Converter_num(x)

Conversion from decimal to binary and viceversa
Binary representation of 26 : 0b11010
Decimal representation of 0b11010 : 26 

Conversion from decimal to Hexadecimal and viceversa
Hexadecimal representation of 26 : 0x1a
Decimal representation of 0x1a : 26 

Conversion from Hexadecimal to binary
Binary representation of 0x1a : 0b11010 

Conversion from binary to Hexadecimal
Hexadecimal representation of 0b11010 : 0x1a 



2\. Write a function that converts a 32 bit word into a single precision floating point (i.e. interprets the various bits as sign, mantissa and exponent)

In [1]:
def word_to_float32(bit):
   
    mantissa1 = bit[0:23]
    exponent1 = bit[23:31]
    sign1 = bit[31]
    
    mantissa = 1
    power = -1
    
    #calculate mantissa
    for x in mantissa1:
        mantissa += (int(x) * pow(2, power))
        power -= 1 
    
    #calculate exponent
    exponent_bias = int(exponent1, 2)
    exponent_unbias = exponent_bias - 127
    
    #determine the sign
    if sign1 == '0':
        sign = 1
    else:
        sign = -1
    
    #convert 32 bit to  a single precision floating point 
    single_prec_flo_num = sign * mantissa * pow(2, exponent_unbias)
    
    #print the results
    print('Sign:',sign1, ', Exponent:',exponent1, ', Mantissa:',mantissa1)
    print('Sign:',sign, ', Exponent:',exponent_bias, ', Mantissa:',mantissa)
    print('Float Number:', single_prec_flo_num)
    
#call the function
word_to_float32('10100000100000000001000000000011')

Sign: 1 , Exponent: 00000001 , Mantissa: 10100000100000000001000
Sign: -1 , Exponent: 1 , Mantissa: 1.6269540786743164
Float Number: -1.9124753285289384e-38


3\. Write a program to determine the underflow and overflow limits (within a factor of 2) for python on your computer. 

**Tips**: define two variables inizialized to 1 and halve/double them enough time to exceed the under/over-flow limits  

In [7]:
import math

underflow = 1.0
overflow = 1.0
counter_overflow = 0;
counter_underflow = 0;

#multipling overflow with power of 2 until the maximum value
while(overflow * 2 < math.inf):
    overflow = overflow * 2
    counter_overflow += 1
    
print('Overflow:','\n', counter_overflow,'\n')
print('Maximum: ' + str(overflow),'\n')

#mdeviding underflow by 2 until the minimum value
while(underflow /2 > 0):
    underflow = underflow / 2
    counter_underflow += 1
    
print('Underflow:','\n',counter_underflow, '\n')
print('Minimum: ' + str(underflow),'\n') 


Overflow: 
 1023 

Maximum: 8.98846567431158e+307 

Underflow: 
 1074 

Minimum: 5e-324 



4\. Write a program to determine the machine precision

**Tips**: define a new variable by adding a smaller and smaller value (proceeding similarly to prob. 2) to an original variable and check the point where the two are the same 

In [3]:

x = 150
y = 0
c = 2.0
counter = 0

while x != y:
    counter += 1
    y = x + c
    c /= 2
    x += c
        
print("Sum:", counter)
print("Final x:", x)
print("No Effect on C:", c)
print("Machine precision: " + str(c*2))


Sum: 47
Final x: 152.0
No Effect on C: 1.4210854715202004e-14
Machine precision: 2.842170943040401e-14


5\. Write a function that takes in input three parameters $a$, $b$ and $c$ and prints out the two solutions to the quadratic equation $ax^2+bx+c=0$ using the standard formula:
$$
x=\frac{-b\pm\sqrt{b^2-4ac}}{2a}
$$

(a) use the program to compute the solution for $a=0.001$, $b=1000$ and $c=0.001$

(b) re-express the standard solution formula by multiplying top and bottom by $-b\mp\sqrt{b^2-4ac}$ and again find the solution for $a=0.001$, $b=1000$ and $c=0.001$. How does it compare with what previously obtained? Why?

(c) write a function that compute the roots of a quadratic equation accurately in all cases

In [17]:
import math

a = 0.001
b = 1000
c = 0.001


#a) compute roots by the first method:

def method1 (a, b, c):
    x1 = (-b - math.sqrt(b*b - 4*a*c)) / (2*a)
    x2 = (-b + math.sqrt(b*b - 4*a*c)) / (2*a)
    return x1, x2

from numpy import roots
roots_2 = roots([0.001,1000,0.001])

print ('a)')
print ('The correct roots should be =======> x1 = ', roots_2[0], 'x2 =', roots_2[1])
print ('The roots acquired from method 1:' , method1 (a, b, c ),'\n')
#in methid 1: X1 is accurate, but X2 is not accurate



#b) compute solutions by the second method:

def method2 (a, b, c):
    x1 = 2*c / (-b + math.sqrt(b*b - 4*a*c))
    x2 = 2*c / (-b - math.sqrt(b*b - 4*a*c))
    return x1, x2

print ('b)')
print ('The roots acquired from method 2:' , method2 (a, b, c ),'\n')
#in method 2: X1 is not accurate, however X2 is accurate

#The result of method 1 is different from the result of method 2, due to changing the order of operations that changes the result
#So, we can take the best solution of each method and use them in the third method with an optimal precision



#c) compute the more accurate roots:

def method3(a, b, c):
    x1 = (-b - math.sqrt(b*b - 4*a*c)) / (2*a)
    x2 = 2*c / (-b - math.sqrt(b*b - 4*a*c))
    return x1, x2

print ('c)')
print ('The roots acquired from method 3:' , method3 (a, b, c ),'\n')
print ('Method 3 is the most accurate method with an optimal precision.')


a)
The correct roots should be =======> x1 =  -999999.999999 x2 = -1.000000000001e-06
The roots acquired from method 1: (-999999.999999, -9.999894245993346e-07) 

b)
The roots acquired from method 2: (-1000010.5755125057, -1.000000000001e-06) 

c)
The roots acquired from method 3: (-999999.999999, -1.000000000001e-06) 

Method 3 is the most accurate method with an optimal precision.


6\. Write a program that implements the function $f(x)=x(x−1)$

(a) Calculate the derivative of the function at the point $x = 1$ using the derivative definition:

$$
\frac{{\rm d}f}{{\rm d}x} = \lim_{\delta\to0} \frac{f(x+\delta)-f(x)}{\delta}
$$

with $\delta = 10^{−2}$. Calculate the true value of the same derivative analytically and compare with the answer your program gives. The two will not agree perfectly. Why not?

(b) Repeat the calculation for $\delta = 10^{−4}, 10^{−6}, 10^{−8}, 10^{−10}, 10^{−12}$ and $10^{−14}$. How does the accuracy scales with $\delta$?

In [16]:
#a) compute the function value:

df_real = 1
x = 1

def f(x):
    result_f = x * (x - 1)
    return result_f

def df(x, i):
    delta = 10**(-i)
    result_df = (f(x + delta) - f(x))/delta
    return result_df

acquired_value_2 = df(x,2)

print ('a)')

#print the analytical derivative value
print('Real Value of Derivative: ', df_real)

#print the acquired derivative value
print ('Acquired Value of Derivative for i = 2 is:', acquired_value_2)

# The results are not identical because we are using the a taylor expansion (incremental ratio).
# So, this method mathematically has an error.

#check the accuracy of this method
def accuracy(df, df_real):
    result_accuracy = (abs(df - df_real)/df_real*100)
    return result_accuracy

print('Accuracy:', accuracy(acquired_value_2, df_real),'\n')


#b)

print ('b)')
print('Real Value of Derivative: ', df_real)

for i in range(4,15): 
    if i%2==0:
        list = i
        print('Acquired Value of Derivative for i =', i, 'is: ', df(x,i))
        acquired_value_i = df(x,i)
        print('Accuracy for i =', i, 'is: ', accuracy(acquired_value_i, df_real),'\n')

#At first, by decreasing delta the accuracy increases. But then it gets worse due to the subtraction of similiar numbers.        


a)
Real Value of Derivative:  1
Acquired Value of Derivative for i = 2 is: 1.010000000000001
Accuracy: 1.0000000000000897 

b)
Real Value of Derivative:  1
Acquired Value of Derivative for i = 4 is:  1.0000999999998899
Accuracy for i = 4 is:  0.009999999988985486 

Acquired Value of Derivative for i = 6 is:  1.0000009999177333
Accuracy for i = 6 is:  9.99917733279787e-05 

Acquired Value of Derivative for i = 8 is:  1.0000000039225287
Accuracy for i = 8 is:  3.9225287462585356e-07 

Acquired Value of Derivative for i = 10 is:  1.000000082840371
Accuracy for i = 10 is:  8.284037100736441e-06 

Acquired Value of Derivative for i = 12 is:  1.0000889005833413
Accuracy for i = 12 is:  0.008890058334132256 

Acquired Value of Derivative for i = 14 is:  0.9992007221626509
Accuracy for i = 14 is:  0.07992778373491216 



7\. Consider the integral of the semicircle of radius 1:
$$
I=\int_{-1}^{1} \sqrt(1-x^2) {\rm d}x
$$
which it's known to be $I=\frac{\pi}{2}=1.57079632679...$.
Alternatively we can use the Riemann definition of the integral:
$$
I=\lim_{N\to\infty} \sum_{k=1}^{N} h y_k 
$$

with $h=2/N$ the width of each of the $N$ slices the domain is divided into and where
$y_k$ is the value of the function at the $k-$th slice.

(a) Write a programe to compute the integral with $N=100$. How does the result compares to the true value?

(b) How much can $N$ be increased if the computation needs to be run in less than a second? What is the gain in running it for 1 minute? 


In [55]:
from math import sqrt, pi
import timeit

#a)

true_I = pi/2
N=100
h=2/N

def area_function(n):
    b = 2 / n
    x = -1
    I = 0
    
    while(x<1):
        I += ((1-(x**2))**.5)*b
        x += b
    return I

calculated_I = I

print('a)  For N = 100:')
print('The true value of semi pi is: ', true_I)
print('The calculated value of semi pi is: ', calculated_I)
print('The error = true_I - calculated_I  is: ',true_I-calculated_I)
print('The accuracy = ', abs((calculated_I - true_I)/true_I*100),'\n')

start_time = timeit.default_timer() #set the initial time for the evaluation
result = area_function(N)
end_time = timeit.default_timer() # set the time after the checking cycle
total_time = end_time - start_time

print('Time required for N = 100: ', total_time, 's')
print ('Start time:', start_time)
print ('End time:', end_time, '\n')


a)  For N = 100:
The true value of semi pi is:  1.5707963267948966
The calculated value of semi pi is:  1.5691342555492505
The error = true_I - calculated_I  is:  0.0016620712456461018
The accuracy =  0.10581074180619238 

Time required for N = 100:  0.0007483000008505769 s
Start time: 9367.2901138
End time: 9367.2908621 



In [62]:

#b) For 1 s

from math import sqrt, pi
import timeit

# 1s

def area_function(n):
    b = 2 / n
    x = -1
    area = 0
    
    while(x<1):
        area += ((1-(x**2))**.5)*b
        x += b
    return area

N = 522000
start_time = timeit.default_timer() #set the initial time for the evaluation
result = area_function(N)
end_time = timeit.default_timer() # set the time after the checking cycle
total_time = end_time - start_time

print('b)  For 1 s:')
print('Time required for N = 522000 : ', total_time, 's')
print ('Start time:', start_time)
print ('End time:', end_time, '\n')


b)  For 1 s:
Time required for N = 522000 :  1.0177712999993673 s
Start time: 9862.7617394
End time: 9863.7795107 



In [ ]:

#for 60 sec

from math import sqrt, pi
import timeit

# 60s

def area_function(n):
    b = 2 / n
    x = -1
    area = 0
    
    while(x<1):
        area += ((1-(x**2))**.5)*b
        x += b
    return area

N = 27400000

start_time = timeit.default_timer() #set the initial time for the evaluation
result = area_function(N)
end_time = timeit.default_timer() # set the time after the checking cycle
total_time = end_time - start_time

print('b)  For 60 s:')
print('Time required for N = 27400000: ', total_time, 's')
print ('Start time:', start_time)
print ('End time:', end_time, '\n')

